# 基础组件之Trainer

# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Generating train split: 0 examples [00:00, ? examples/s]/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 7766 examples [00:00, 230608.33 examples/s]
Filter: 100%|██████████| 7766/7766 [00:00<00:00, 469486.38 examples/s]


Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 划分数据集

In [3]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step4 数据集预处理

In [4]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")


def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples


tokenized_datasets = datasets.map(
    process_function, batched=True, remove_columns=datasets["train"].column_names
)
tokenized_datasets

Map: 100%|██████████| 777/777 [00:00<00:00, 20117.62 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

## Step5 创建模型

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.config

BertConfig {
  "_name_or_path": "hfl/rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

## **Step6 创建评估函数

In [7]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [8]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## **Step7 创建TrainingArguments

In [9]:
train_args = TrainingArguments(
    output_dir="./checkpoints",  # 输出文件夹
    per_device_train_batch_size=64,  # 训练时的batch_size
    per_device_eval_batch_size=128,  # 验证时的batch_size
    logging_steps=10,  # log 打印的频率
    evaluation_strategy="epoch",  # 评估策略
    save_strategy="epoch",  # 保存策略
    save_total_limit=3,  # 最大保存数
    learning_rate=2e-5,  # 学习率
    weight_decay=0.01,  # weight_decay
    metric_for_best_model="f1",  # 设定评估指标
    load_best_model_at_end=True,
)  # 训练完成后加载最优模型
train_args

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
f

## **Step8 创建Trainer

In [10]:
from transformers import DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric,
)

## **Step9 模型训练

In [11]:
trainer.train()

  3%|▎         | 11/330 [00:02<00:51,  6.20it/s]

{'loss': 0.646, 'grad_norm': 3.5232670307159424, 'learning_rate': 1.9393939393939395e-05, 'epoch': 0.09}


  6%|▋         | 21/330 [00:03<00:48,  6.39it/s]

{'loss': 0.546, 'grad_norm': 2.0024499893188477, 'learning_rate': 1.8787878787878792e-05, 'epoch': 0.18}


  9%|▉         | 31/330 [00:05<00:47,  6.31it/s]

...

                                                 
100%|██████████| 330/330 [00:54<00:00,  6.45it/s]

{'eval_loss': 0.27881765365600586, 'eval_accuracy': 0.8854568854568855, 'eval_f1': 0.9154795821462488, 'eval_runtime': 0.6033, 'eval_samples_per_second': 1288.003, 'eval_steps_per_second': 11.604, 'epoch': 3.0}


100%|██████████| 330/330 [00:55<00:00,  5.99it/s]

{'train_runtime': 55.1018, 'train_samples_per_second': 380.459, 'train_steps_per_second': 5.989, 'train_loss': 0.29492162068684896, 'epoch': 3.0}


TrainOutput(global_step=330, training_loss=0.29492162068684896, metrics={'train_runtime': 55.1018, 'train_samples_per_second': 380.459, 'train_steps_per_second': 5.989, 'total_flos': 351909933963264.0, 'train_loss': 0.29492162068684896, 'epoch': 3.0})

## **Step10 模型评估

In [12]:
trainer.evaluate(tokenized_datasets["test"])

100%|██████████| 7/7 [00:00<00:00, 14.06it/s]


{'eval_loss': 0.27881765365600586,
 'eval_accuracy': 0.8854568854568855,
 'eval_f1': 0.9154795821462488,
 'eval_runtime': 0.6054,
 'eval_samples_per_second': 1283.45,
 'eval_steps_per_second': 11.563,
 'epoch': 3.0}

## **Step11 模型预测

In [13]:
trainer.predict(tokenized_datasets["test"])

100%|██████████| 7/7 [00:00<00:00, 14.17it/s]


PredictionOutput(predictions=array([[ 1.6659021, -1.5507957],
       [-1.6304892,  1.9699897],
       [ 1.8136438, -1.4618142],
       ...,
       [ 1.3355892, -1.5159024],
       [-2.6307316,  2.9208736],
       [-2.280545 ,  2.4365335]], dtype=float32), label_ids=array([0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
    

In [14]:
from transformers import pipeline

id2_label = id2_label = {0: "差评！", 1: "好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [15]:
sen = "我觉得不错！"
pipe(sen)

[{'label': '好评！', 'score': 0.9911233186721802}]